# Getting Started

This notebook is for running all post-hoc tests. Plots will NOT be generated by this notebook. Please see the `significance_visualization.ipynb` notebook for plot generation. This notebook creates all the require pickle files for the `significance_visualization.ipynb` notebook to generate plots!

# Imports and Utils

### Notes

`DataFrameIndexWrapper` servers as a wrapper for a pandas multi-indexing DataFrame. It simply helps to format the data into the proper format using the `get_group_data()` method when conducting post-hoc tests.

The `debug_groupby()` function aims to display all the DataFrame groupings found by the `.groupby()` method. Useful for debugging.

In [1]:
import os
import glob
import pickle
from pathlib import Path

import seaborn as sns  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 200)
from tqdm import tqdm 

from IPython.core.debugger import set_trace

import directory_structure as ds
from analysis_utils import DataFrameIndexWrapper, debug_groupby

%load_ext autoreload
%autoreload 2

# Data

There are two versions of the classification results for analysis. The initial data reported in the paper and replication classification which helped to confirm the results were reproducible.

- Paper results are stored in the `v2-ErrP-variations-March-3-2022.tar.xz` file. By default, `current_analysis` will use the directory from this decompressed file.


- Reproducible results are stored in `v2-replication.tar.xz` file. These results are produced by using the same experiments as the paper results but act as a second run. Due to random operations contained within TensorFlow (not being able to be controlled by a seed), this makes the EEGNet results stochastic even when all seeds are set. Thus, replication results act as our second run to help validate the EEGNet results.
    - To use these results you'll have to change the `current_analysis` variable to the string `v2-replication` which will make this notebook use said directory instead.

In [2]:
current_analysis = 'v2-ErrP-variations-March-3-2022'

# Example of using replication results, uncomment to use.
# current_analysis = 'v2-replication'

## Download 

The below class attempts to automatically download the paper results and replication results into the `../scripts/classification/exps` directory. Additionally, it will extract the tar files for you as well.

Alternatively, you can manually download and extract the files yourself using the below links. Make sure to extract the tar files to the `../scripts/classification/exps` directory, otherwise this notebook's code will fail to find the results.

- [Paper Results](https://drive.google.com/uc?export=download&id=14IctHzHOm1UTeq4fO8DIep0UoVJP8dUY)

- [Replication Results](https://drive.google.com/uc?export=download&id=11GIog1bpSASE-sUFjvq24MWcQa1h2bdl)

In [3]:
from analysis_utils import DownloadResults
result_info = dict(           
    paper_results=dict(
        file="v2-ErrP-variations-March-3-2022.tar.xz",
        directory='v2-ErrP-variations-March-3-2022',
        url="https://drive.google.com/uc?export=download&id=14IctHzHOm1UTeq4fO8DIep0UoVJP8dUY",
    ),
    replication=dict(
        file="v2-replication.tar.xz",
        directory="v2-replication",
        url="https://drive.google.com/uc?export=download&id=11GIog1bpSASE-sUFjvq24MWcQa1h2bdl",
    )
)
save_dir = Path(ds.ROOT_DIR) / ds.EXPS_DIR
download_results = DownloadResults(result_info, save_dir)
download_results()

Skipping paper_results, already exists
Skipping replication, already exists
Done


# File Loading

Below is an example of loading the "test-metrics.csv" file generated by LOGOCV classification. here `ds.ROOT_DIR` provides the absolute path to the classification directory while `ds.EXPS_DIR` specifies the name of the experiment directory (typically denoted as "exps") which holds all the experiment results.

Note: If the `v2-ErrP-variations-March-3-2022/` directory does not exist, make sure to decompress the `v2-ErrP-variations-March-3-2022.tar.xz` file within the `deep-bci/scripts/classification/exps` directory.

In [4]:
target_dir = os.path.join(current_analysis, 'logocv-v2-eegnet-async-upsample')

target_file = '**/test-metrics.csv'

parent_dir = Path(ds.ROOT_DIR) / ds.EXPS_DIR

target_path = parent_dir / target_dir / target_file

target_path

PosixPath('/home/dev/mnt/scripts/classification/exps/v2-ErrP-variations-March-3-2022/logocv-v2-eegnet-async-upsample/**/test-metrics.csv')

Below, we search over all the files in the `target_path` extracting all the "test-metrics.csv" files. Notice that if we have 9 subjects and 4 sub-tasks then there should be a total of 36 files as we conduct 1 LOGOCV for each subject and sub-task pairing.

In [5]:
test_metric_files = glob.glob(str(target_path), recursive=True)

print(f"Found {len(test_metric_files)} files")

subjects = 9 
tasks = 4
estimated_files = subjects * tasks
assert estimated_files == len(test_metric_files), f"Number of files found {len(test_metric_files)} does not match estimated {estimated_files}"

Found 36 files


#  DataFrame Building

Below we provide an example of loading all the files into Pandas DataFrames. We additionally perform some post-processing such as converting types and expanding some columns into multiple columns. 

For instance, we expand the "exp" column into three new columns denoted 'tst_dataset', 'tst_subject', and 'tst_trial'. These represent the dataset, subject, and trial selected by LOGOCV for testing at a particular iteration. Thus, 'BGSObs-8-1' indicates that LOGOCV was performed on BGSObs, subject 8, and the test set was trial 1 (trial is also sometimes referred to as a "block"). In other words, this means trials 2-10 were used as the training data. Thus, it is easier to refer to each LOGOCV iteration by its test trial.

The LOGOCV test trial AND the other testing datasets used to test transfer performance are given in the 'dataset', 'subject', and 'trial' columns.

In [6]:
rename_cols = {
    'tpr': 'TPR',
    'tnr': 'TNR',
    'ppv': 'PPV',
    'nll': 'NLL'
}

convert_type = {
    'subject': str,
    'trial': str,
}

exp_cols = ['tst_dataset', 'tst_subject', 'tst_trial']

mi = ['tst_dataset', 'tst_subject', 'tst_trial', 
      'dataset', 'subject', 'trial']



def parse_exp(df, exp_cols):
    exp_values = df.exp.str.split("-", expand=True)
    # Insert exp columns at front of dataframe
    for col, index in zip(exp_cols[::-1], range(len(exp_cols))[::-1]):
        # print(f"Column: {col} Index: {index}")
        df.insert(0, col, exp_values[index])
    
    df.drop('exp', axis=1, inplace=True)
    return df

def convert_types(df, convert_type):
    for col, t in convert_type.items():
        df[col] = df[col].astype(t)
    return df

# Load all files in list_files
tm_dfs = []
for tm_file in test_metric_files:
    tm_df = pd.read_csv(tm_file)
    convert_types(tm_df, convert_type)
    tm_df = parse_exp(tm_df, exp_cols)
    tm_df.rename(columns=rename_cols, inplace=True)
    tm_dfs.append(tm_df)

In [7]:
tm_df = pd.concat(tm_dfs)
tm_df = tm_df.set_index(mi).sort_index()
tm_dfw = DataFrameIndexWrapper(tm_df)
display(tm_dfw)

DataFrameIndexWrapper(shape=(11160, 9)

Example of indexing using `__getitem__`. Notice each multi-index column needs to index in order and wrapper using a list.

In [8]:
tm_dfw[['OAOut'], ['1'], ['1']][['bACC', 'TPR']]

bACC       TPR
tst_dataset tst_subject tst_trial dataset subject trial                    
OAOut       1           1         BGSInt  1       1      0.634871  0.285714
                                                  10     0.577451  0.200000
                                                  2      0.594370  0.217391
                                                  3      0.667072  0.368421
                                                  4      0.574521  0.173913
                                                  5      0.622827  0.285714
                                                  6      0.631589  0.307692
                                                  7      0.627828  0.285714
                                                  8      0.652718  0.333333
                                                  9      0.602456  0.250000
                                  BGSObs  1       1      0.671919  0.400000
                                                  10     0.615321  0.260870
                                                  2      0.504802  0.040000
                                                  3      0.599789  0.225806
                                                  4      0.545051  0.105263
                                                  5      0.519908  0.050000
                                                  6      0.727203  0.500000
                                                  7      0.538542  0.130435
                                                  8      0.529981  0.090909
                                                  9      0.535112  0.103448
                                  OAObs   1       1      0.730409  0.500000
                                                  10     0.625169  0.307692
                                                  2      0.688458  0.416667
                                                  3      0.684524  0.416667
                                                  4      0.582143  0.200000
                                                  5      0.675840  0.384615
                                                  6      0.853187  0.750000
                                                  7      0.620272  0.300000
                                                  8      0.608943  0.266667
                                                  9      0.730464  0.500000
                                  OAOut   1       1      0.930791  0.900000

Example indexing using custom `loc()` method. Notice `*args` correspond to non-multi-indexes while multi-indexes are specified using `**kwargs`.

In [9]:
tm_dfw.loc('bACC', 'TPR', tst_dataset="OAOut", tst_subject='1', tst_trial='1')

bACC       TPR
tst_dataset tst_subject tst_trial dataset subject trial                    
OAOut       1           1         BGSInt  1       1      0.634871  0.285714
                                                  10     0.577451  0.200000
                                                  2      0.594370  0.217391
                                                  3      0.667072  0.368421
                                                  4      0.574521  0.173913
                                                  5      0.622827  0.285714
                                                  6      0.631589  0.307692
                                                  7      0.627828  0.285714
                                                  8      0.652718  0.333333
                                                  9      0.602456  0.250000
                                  BGSObs  1       1      0.671919  0.400000
                                                  10     0.615321  0.260870
                                                  2      0.504802  0.040000
                                                  3      0.599789  0.225806
                                                  4      0.545051  0.105263
                                                  5      0.519908  0.050000
                                                  6      0.727203  0.500000
                                                  7      0.538542  0.130435
                                                  8      0.529981  0.090909
                                                  9      0.535112  0.103448
                                  OAObs   1       1      0.730409  0.500000
                                                  10     0.625169  0.307692
                                                  2      0.688458  0.416667
                                                  3      0.684524  0.416667
                                                  4      0.582143  0.200000
                                                  5      0.675840  0.384615
                                                  6      0.853187  0.750000
                                                  7      0.620272  0.300000
                                                  8      0.608943  0.266667
                                                  9      0.730464  0.500000
                                  OAOut   1       1      0.930791  0.900000

Example of `get_group_data()` flattening the data by grouping by 'tst_dataset' and 'dataset' while grabbing the 'bACC' score. By grouping by 'tst_dataset' and 'dataset' this returns all transfer combinations (source $\rightarrow $ target and source $\rightarrow$ source) for all subjects and trials.

In [10]:
group_data = tm_dfw.get_group_data(['tst_dataset', 'dataset'], score='bACC')
group_data

,score
group,
BGSInt-BGSInt,0.836468
BGSInt-BGSInt,0.757354
BGSInt-BGSInt,0.798863
BGSInt-BGSInt,0.806560
BGSInt-BGSInt,0.855242
...,...
OAOut-OAOut,0.882936
OAOut-OAOut,0.825867
OAOut-OAOut,0.873662


This is an example of indexing the BGSInt $\rightarrow$ BGSObs transfer data. Notice there are 900 bACC scores (9 subjects * 10 LOGOCV trials * 10 additional test per LOGOCV trial ).

In [11]:
group_data.loc['BGSInt-BGSObs']

,score
group,
BGSInt-BGSObs,0.765794
BGSInt-BGSObs,0.729411
BGSInt-BGSObs,0.801884
BGSInt-BGSObs,0.699750
BGSInt-BGSObs,0.724346
...,...
BGSInt-BGSObs,0.553710
BGSInt-BGSObs,0.572693
BGSInt-BGSObs,0.582724


# Mean/Std Analysis

Below resides a manual analysis of mean and STDs. You can do compute additional metrics if you'd like.

### Single Trial Example

Looks at transfer for OAObs $\rightarrow$ BGSInt for subject 1 and ONLY the LOGOCV test trial 10 (this means trials 1-9 were used for training).

In [12]:
tar_df = tm_dfw.loc(
    tst_dataset = "OAObs", 
    tst_subject = "1",
    tst_trial='10',
    dataset='BGSInt',
    subject='1')
tar_df

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           10        BGSInt  1       1      0.795913  0.876442   
                                                  10     0.602714  0.948106   
                                                  2      0.789601  0.881573   
                                                  3      0.678968  0.880413   
                                                  4      0.622998  0.892293   
                                                  5      0.682457  0.884484   
                                                  6      0.801109  0.908421   
                                                  7      0.669734  0.907299   
                                                  8      0.589920  0.950584   
                                                  9      0.662540  0.942766   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           10        BGSInt  1       1      0.714286  0.038023   
                                                  10     0.250000  0.055556   
                                                  2      0.695652  0.059701   
                                                  3      0.473684  0.037190   
                                                  4      0.347826  0.035556   
                                                  5      0.476190  0.042553   
                                                  6      0.692308  0.050279   
                                                  7      0.428571  0.034884   
                                                  8      0.222222  0.048193   
                                                  9      0.375000  0.088235   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           10        BGSInt  1       1      0.877541  0.072202   
                                                  10     0.955427  0.090909   
                                                  2      0.883549  0.109966   
                                                  3      0.884252  0.068966   
                                                  4      0.898170  0.064516   
                                                  5      0.888724  0.078125   
                                                  6      0.909910  0.093750   
                                                  7      0.910896  0.064516   
                                                  8      0.957618  0.079208   
                                                  9      0.950081  0.142857   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           10        BGSInt  1       1      0.270820  0.181410   
                                                  10     0.170668  0.078508   
                                                  2      0.258696  0.159882   
                                                  3      0.284645  0.088830   
                                                  4      0.251652  0.050719   
                                                  5      0.261638  0.120359   
                                                  6      0.252477  0.102445   
                                                  7      0.200059  0.073020   
                                                  8      0.124632  0.092650   
                                                  9      0.136332  0.111984   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           10        BGSInt  1       1      0.887325  
                                                  10     0.64

In [13]:
tar_df.mean()

bACC    0.689595
ACC     0.907238
TPR     0.467574
PPV     0.049017
TNR     0.911617
F1      0.086501
NLL     0.221162
PR      0.105981
ROC     0.803692
dtype: float64

### All Trials

Looks at transfer for OAObs $\rightarrow$ BGSInt for subject 1 across all LOGOCV trials. Notice `debug_groupby()` will print out all the individual groups used to compute the metric.

In [14]:
tar_df = tm_dfw.loc(
    tst_dataset = "OAObs", 
    tst_subject = "1", 
    dataset='BGSInt',
    subject='1')

grps = tar_df.groupby(level=['tst_trial'])

grps.mean()

,bACC,ACC,TPR,PPV,TNR,F1,NLL,PR,ROC
tst_trial,,,,,,,,,
1,0.694439,0.882226,0.502835,0.041577,0.886043,0.074652,0.270006,0.106285,0.805494
10,0.689595,0.907238,0.467574,0.049017,0.911617,0.086501,0.221162,0.105981,0.803692
2,0.692271,0.913636,0.466442,0.052583,0.918099,0.091230,0.211394,0.113685,0.812707
3,0.674495,0.914561,0.429569,0.046919,0.919421,0.082404,0.212525,0.105199,0.788551
4,0.703720,0.915951,0.487190,0.056996,0.920250,0.098990,0.195938,0.131823,0.844729
5,0.654066,0.920601,0.382199,0.051620,0.925933,0.085096,0.201249,0.100274,0.771719
6,0.727930,0.881269,0.571518,0.047761,0.884343,0.086463,0.255815,0.118104,0.825809
7,0.689352,0.892440,0.482049,0.041486,0.896656,0.074388,0.255686,0.106034,0.791038
8,0.716532,0.903412,0.525882,0.053457,0.907182,0.094879,0.233021,0.118892,0.825173


In [15]:
debug_groupby(grps)

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           1         BGSInt  1       1      0.731884  0.819712   
                                                  10     0.574305  0.940841   
                                                  2      0.742908  0.831733   
                                                  3      0.737796  0.842028   
                                                  4      0.733254  0.855153   
                                                  5      0.692419  0.857562   
                                                  6      0.829501  0.888947   
                                                  7      0.702228  0.901438   
                                                  8      0.558387  0.942614   
                                                  9      0.641707  0.942236   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           1         BGSInt  1       1      0.642857  0.023747   
                                                  10     0.200000  0.039216   
                                                  2      0.652174  0.040000   
                                                  3      0.631579  0.036810   
                                                  4      0.608696  0.044164   
                                                  5      0.523810  0.037671   
                                                  6      0.769231  0.045872   
                                                  7      0.500000  0.037838   
                                                  8      0.166667  0.031250   
                                                  9      0.333333  0.079208   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           1         BGSInt  1       1      0.820910  0.045802   
                                                  10     0.948610  0.065574   
                                                  2      0.833641  0.075377   
                                                  3      0.844014  0.069565   
                                                  4      0.857813  0.082353   
                                                  5      0.861029  0.070288   
                                                  6      0.889772  0.086580   
                                                  7      0.904455  0.070352   
                                                  8      0.950107  0.052632   
                                                  9      0.950081  0.128000   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           1         BGSInt  1       1      0.377413  0.155436   
                                                  10     0.169147  0.080620   
                                                  2      0.359819  0.146946   
                                                  3      0.376174  0.066503   
                                                  4      0.318587  0.117904   
                                                  5      0.314963  0.155937   
                                                  6      0.272995  0.095415   
                                                  7      0.237364  0.057230   
                                                  8      0.134334  0.102629   
                                                  9      0.139266  0.084231   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           1         BGSInt  1       1      0.896556  
                                                  10     0.66

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           10        BGSInt  1       1      0.795913  0.876442   
                                                  10     0.602714  0.948106   
                                                  2      0.789601  0.881573   
                                                  3      0.678968  0.880413   
                                                  4      0.622998  0.892293   
                                                  5      0.682457  0.884484   
                                                  6      0.801109  0.908421   
                                                  7      0.669734  0.907299   
                                                  8      0.589920  0.950584   
                                                  9      0.662540  0.942766   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           10        BGSInt  1       1      0.714286  0.038023   
                                                  10     0.250000  0.055556   
                                                  2      0.695652  0.059701   
                                                  3      0.473684  0.037190   
                                                  4      0.347826  0.035556   
                                                  5      0.476190  0.042553   
                                                  6      0.692308  0.050279   
                                                  7      0.428571  0.034884   
                                                  8      0.222222  0.048193   
                                                  9      0.375000  0.088235   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           10        BGSInt  1       1      0.877541  0.072202   
                                                  10     0.955427  0.090909   
                                                  2      0.883549  0.109966   
                                                  3      0.884252  0.068966   
                                                  4      0.898170  0.064516   
                                                  5      0.888724  0.078125   
                                                  6      0.909910  0.093750   
                                                  7      0.910896  0.064516   
                                                  8      0.957618  0.079208   
                                                  9      0.950081  0.142857   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           10        BGSInt  1       1      0.270820  0.181410   
                                                  10     0.170668  0.078508   
                                                  2      0.258696  0.159882   
                                                  3      0.284645  0.088830   
                                                  4      0.251652  0.050719   
                                                  5      0.261638  0.120359   
                                                  6      0.252477  0.102445   
                                                  7      0.200059  0.073020   
                                                  8      0.124632  0.092650   
                                                  9      0.136332  0.111984   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           10        BGSInt  1       1      0.887325  
                                                  10     0.64

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           2         BGSInt  1       1      0.792525  0.869712   
                                                  10     0.581647  0.955371   
                                                  2      0.769710  0.884774   
                                                  3      0.704539  0.879429   
                                                  4      0.713943  0.902043   
                                                  5      0.685177  0.889868   
                                                  6      0.770331  0.923158   
                                                  7      0.710816  0.918487   
                                                  8      0.568044  0.961743   
                                                  9      0.625973  0.951775   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           2         BGSInt  1       1      0.714286  0.036101   
                                                  10     0.200000  0.054054   
                                                  2      0.652174  0.057915   
                                                  3      0.526316  0.040650   
                                                  4      0.521739  0.056604   
                                                  5      0.476190  0.044643   
                                                  6      0.615385  0.053691   
                                                  7      0.500000  0.045752   
                                                  8      0.166667  0.050000   
                                                  9      0.291667  0.086420   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           2         BGSInt  1       1      0.870765  0.068729   
                                                  10     0.963293  0.085106   
                                                  2      0.887246  0.106383   
                                                  3      0.882762  0.075472   
                                                  4      0.906147  0.102128   
                                                  5      0.894164  0.081633   
                                                  6      0.925278  0.098765   
                                                  7      0.921632  0.083832   
                                                  8      0.969421  0.076923   
                                                  9      0.960279  0.133333   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           2         BGSInt  1       1      0.279337  0.213970   
                                                  10     0.136920  0.051019   
                                                  2      0.253685  0.269210   
                                                  3      0.291978  0.077463   
                                                  4      0.235307  0.104412   
                                                  5      0.239134  0.112444   
                                                  6      0.243980  0.097808   
                                                  7      0.202619  0.056308   
                                                  8      0.106789  0.056371   
                                                  9      0.124192  0.097846   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           2         BGSInt  1       1      0.903091  
                                                  10     0.68

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           3         BGSInt  1       1      0.795913  0.876442   
                                                  10     0.552976  0.947587   
                                                  2      0.729928  0.891175   
                                                  3      0.681204  0.884843   
                                                  4      0.669292  0.898793   
                                                  5      0.663593  0.893784   
                                                  6      0.849639  0.928947   
                                                  7      0.640461  0.919552   
                                                  8      0.536779  0.954304   
                                                  9      0.625168  0.950185   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           3         BGSInt  1       1      0.714286  0.038023   
                                                  10     0.150000  0.034483   
                                                  2      0.565217  0.053942   
                                                  3      0.473684  0.038627   
                                                  4      0.434783  0.046512   
                                                  5      0.428571  0.042056   
                                                  6      0.769231  0.070423   
                                                  7      0.357143  0.034014   
                                                  8      0.111111  0.027778   
                                                  9      0.291667  0.083333   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           3         BGSInt  1       1      0.877541  0.072202   
                                                  10     0.955952  0.056075   
                                                  2      0.894640  0.098485   
                                                  3      0.888723  0.071429   
                                                  4      0.903801  0.084034   
                                                  5      0.898615  0.076596   
                                                  6      0.930048  0.129032   
                                                  7      0.923779  0.062112   
                                                  8      0.962446  0.044444   
                                                  9      0.958669  0.129630   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           3         BGSInt  1       1      0.280057  0.196646   
                                                  10     0.160068  0.077966   
                                                  2      0.249634  0.185980   
                                                  3      0.278229  0.084391   
                                                  4      0.242964  0.056295   
                                                  5      0.251140  0.130762   
                                                  6      0.202431  0.155108   
                                                  7      0.198745  0.072690   
                                                  8      0.122663  0.038445   
                                                  9      0.139323  0.053712   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           3         BGSInt  1       1      0.879858  
                                                  10     0.63

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           4         BGSInt  1       1      0.797366  0.879327   
                                                  10     0.629287  0.951738   
                                                  2      0.744274  0.877000   
                                                  3      0.761393  0.888780   
                                                  4      0.651541  0.906221   
                                                  5      0.682952  0.885463   
                                                  6      0.815682  0.937368   
                                                  7      0.749751  0.925413   
                                                  8      0.536510  0.953773   
                                                  9      0.668445  0.954425   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           4         BGSInt  1       1      0.714286  0.038911   
                                                  10     0.300000  0.070588   
                                                  2      0.608696  0.051095   
                                                  3      0.631579  0.051948   
                                                  4      0.391304  0.045685   
                                                  5      0.476190  0.042918   
                                                  6      0.692308  0.072581   
                                                  7      0.571429  0.056338   
                                                  8      0.111111  0.027397   
                                                  9      0.375000  0.112500   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           4         BGSInt  1       1      0.880445  0.073801   
                                                  10     0.958574  0.114286   
                                                  2      0.879852  0.094276   
                                                  3      0.891207  0.096000   
                                                  4      0.911779  0.081818   
                                                  5      0.889713  0.078740   
                                                  6      0.939057  0.131387   
                                                  7      0.928073  0.102564   
                                                  8      0.961910  0.043956   
                                                  9      0.961889  0.173077   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           4         BGSInt  1       1      0.263115  0.293269   
                                                  10     0.139890  0.090066   
                                                  2      0.268296  0.101116   
                                                  3      0.250209  0.182945   
                                                  4      0.214957  0.109672   
                                                  5      0.242686  0.153021   
                                                  6      0.178385  0.154287   
                                                  7      0.175170  0.074749   
                                                  8      0.109798  0.060230   
                                                  9      0.116875  0.098874   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           4         BGSInt  1       1      0.900844  
                                                  10     0.72

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           5         BGSInt  1       1      0.755601  0.866827   
                                                  10     0.562677  0.966788   
                                                  2      0.721842  0.875171   
                                                  3      0.655136  0.884843   
                                                  4      0.650134  0.903435   
                                                  5      0.649428  0.912384   
                                                  6      0.734785  0.928421   
                                                  7      0.648244  0.935003   
                                                  8      0.571263  0.968119   
                                                  9      0.591553  0.965024   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           5         BGSInt  1       1      0.642857  0.032028   
                                                  10     0.150000  0.060000   
                                                  2      0.565217  0.047101   
                                                  3      0.421053  0.034632   
                                                  4      0.391304  0.044335   
                                                  5      0.380952  0.045977   
                                                  6      0.538462  0.051095   
                                                  7      0.357143  0.042373   
                                                  8      0.166667  0.062500   
                                                  9      0.208333  0.096154   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           5         BGSInt  1       1      0.868345  0.061017   
                                                  10     0.975354  0.085714   
                                                  2      0.878466  0.086957   
                                                  3      0.889220  0.064000   
                                                  4      0.908963  0.079646   
                                                  5      0.917903  0.082051   
                                                  6      0.931108  0.093333   
                                                  7      0.939345  0.075758   
                                                  8      0.975858  0.090909   
                                                  9      0.974772  0.131579   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           5         BGSInt  1       1      0.305903  0.237437   
                                                  10     0.119796  0.074534   
                                                  2      0.283328  0.089371   
                                                  3      0.273051  0.076887   
                                                  4      0.233866  0.103532   
                                                  5      0.217521  0.161036   
                                                  6      0.177639  0.072914   
                                                  7      0.195666  0.057818   
                                                  8      0.097352  0.052565   
                                                  9      0.108367  0.076651   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           5         BGSInt  1       1      0.870073  
                                                  10     0.64

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           6         BGSInt  1       1      0.770986  0.826923   
                                                  10     0.592488  0.927867   
                                                  2      0.769499  0.841792   
                                                  3      0.788689  0.839567   
                                                  4      0.737478  0.863510   
                                                  5      0.692914  0.858541   
                                                  6      0.831356  0.892632   
                                                  7      0.697665  0.892381   
                                                  8      0.640111  0.941020   
                                                  9      0.758119  0.928458   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           6         BGSInt  1       1      0.714286  0.027322   
                                                  10     0.250000  0.038760   
                                                  2      0.695652  0.045070   
                                                  3      0.736842  0.041791   
                                                  4      0.608696  0.046823   
                                                  5      0.523810  0.037931   
                                                  6      0.769231  0.047393   
                                                  7      0.500000  0.034653   
                                                  8      0.333333  0.057143   
                                                  9      0.583333  0.100719   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           6         BGSInt  1       1      0.827686  0.052632   
                                                  10     0.934976  0.067114   
                                                  2      0.843346  0.084656   
                                                  3      0.840537  0.079096   
                                                  4      0.866260  0.086957   
                                                  5      0.862018  0.070740   
                                                  6      0.893482  0.089286   
                                                  7      0.895330  0.064815   
                                                  8      0.946888  0.097561   
                                                  9      0.932904  0.171779   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           6         BGSInt  1       1      0.358219  0.130959   
                                                  10     0.186165  0.049797   
                                                  2      0.315903  0.152990   
                                                  3      0.353582  0.061419   
                                                  4      0.286139  0.127888   
                                                  5      0.303092  0.142226   
                                                  6      0.229400  0.197900   
                                                  7      0.225625  0.082597   
                                                  8      0.140984  0.116519   
                                                  9      0.159040  0.118749   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           6         BGSInt  1       1      0.899668  
                                                  10     0.69

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           7         BGSInt  1       1      0.841447  0.825962   
                                                  10     0.549043  0.939803   
                                                  2      0.774813  0.852309   
                                                  3      0.687897  0.846457   
                                                  4      0.659203  0.878830   
                                                  5      0.674792  0.869310   
                                                  6      0.801374  0.908947   
                                                  7      0.743041  0.912094   
                                                  8      0.560265  0.946334   
                                                  9      0.601651  0.944356   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           7         BGSInt  1       1      0.857143  0.032258   
                                                  10     0.150000  0.029412   
                                                  2      0.695652  0.048193   
                                                  3      0.526316  0.031949   
                                                  4      0.434783  0.038760   
                                                  5      0.476190  0.037594   
                                                  6      0.692308  0.050562   
                                                  7      0.571429  0.047904   
                                                  8      0.166667  0.033708   
                                                  9      0.250000  0.064516   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           7         BGSInt  1       1      0.825750  0.062176   
                                                  10     0.948086  0.049180   
                                                  2      0.853974  0.090141   
                                                  3      0.849478  0.060241   
                                                  4      0.883623  0.071174   
                                                  5      0.873393  0.069686   
                                                  6      0.910440  0.094241   
                                                  7      0.914654  0.088398   
                                                  8      0.953863  0.056075   
                                                  9      0.953301  0.102564   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           7         BGSInt  1       1      0.363420  0.227061   
                                                  10     0.179469  0.071371   
                                                  2      0.331384  0.119486   
                                                  3      0.358331  0.096848   
                                                  4      0.293359  0.098464   
                                                  5      0.294945  0.117085   
                                                  6      0.231480  0.145740   
                                                  7      0.234357  0.059725   
                                                  8      0.133376  0.035141   
                                                  9      0.136742  0.089419   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           7         BGSInt  1       1      0.893134  
                                                  10     0.63

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           8         BGSInt  1       1      0.756569  0.868750   
                                                  10     0.597470  0.937727   
                                                  2      0.694095  0.862826   
                                                  3      0.780506  0.875000   
                                                  4      0.705027  0.884401   
                                                  5      0.755617  0.889378   
                                                  6      0.837716  0.905263   
                                                  7      0.711353  0.919552   
                                                  8      0.643062  0.946865   
                                                  9      0.683910  0.944356   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           8         BGSInt  1       1      0.642857  0.032491   
                                                  10     0.250000  0.045455   
                                                  2      0.521739  0.039867   
                                                  3      0.684211  0.049808   
                                                  4      0.521739  0.048000   
                                                  5      0.619048  0.056277   
                                                  6      0.769231  0.053476   
                                                  7      0.500000  0.046358   
                                                  8      0.333333  0.063830   
                                                  9      0.416667  0.099010   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           8         BGSInt  1       1      0.870281  0.061856   
                                                  10     0.944940  0.076923   
                                                  2      0.866451  0.074074   
                                                  3      0.876801  0.092857   
                                                  4      0.888315  0.087912   
                                                  5      0.892186  0.103175   
                                                  6      0.906200  0.100000   
                                                  7      0.922705  0.084848   
                                                  8      0.952790  0.107143   
                                                  9      0.951154  0.160000   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           8         BGSInt  1       1      0.293680  0.193590   
                                                  10     0.164518  0.086357   
                                                  2      0.300919  0.141740   
                                                  3      0.307430  0.085341   
                                                  4      0.273336  0.121059   
                                                  5      0.264472  0.150961   
                                                  6      0.251463  0.120687   
                                                  7      0.209437  0.090104   
                                                  8      0.124582  0.093839   
                                                  9      0.140374  0.105241   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           8         BGSInt  1       1      0.901397  
                                                  10     0.69

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           9         BGSInt  1       1      0.869278  0.881250   
                                                  10     0.609793  0.962117   
                                                  2      0.788907  0.880201   
                                                  3      0.738555  0.895177   
                                                  4      0.679850  0.919684   
                                                  5      0.687897  0.895252   
                                                  6      0.855734  0.941053   
                                                  7      0.718599  0.933937   
                                                  8      0.627355  0.970244   
                                                  9      0.633756  0.967144   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           9         BGSInt  1       1      0.857143  0.046693   
                                                  10     0.250000  0.079365   
                                                  2      0.695652  0.059041   
                                                  3      0.578947  0.050926   
                                                  4      0.434783  0.058824   
                                                  5      0.476190  0.046948   
                                                  6      0.769231  0.084034   
                                                  7      0.500000  0.056452   
                                                  8      0.277778  0.104167   
                                                  9      0.291667  0.134615   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           9         BGSInt  1       1      0.881413  0.088561   
                                                  10     0.969586  0.120482   
                                                  2      0.882163  0.108844   
                                                  3      0.898162  0.093617   
                                                  4      0.924918  0.103627   
                                                  5      0.899604  0.085470   
                                                  6      0.942236  0.151515   
                                                  7      0.937198  0.101449   
                                                  8      0.976931  0.151515   
                                                  9      0.975845  0.184211   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
OAObs       1           9         BGSInt  1       1      0.275726  0.224758   
                                                  10     0.130044  0.047584   
                                                  2      0.258976  0.121918   
                                                  3      0.252054  0.117327   
                                                  4      0.214201  0.127869   
                                                  5      0.221994  0.174555   
                                                  6      0.138058  0.209228   
                                                  7      0.176629  0.066513   
                                                  8      0.089013  0.051734   
                                                  9      0.098076  0.118849   

                                                              ROC  
tst_dataset tst_subject tst_trial dataset subject trial            
OAObs       1           9         BGSInt  1       1      0.904854  
                                                  10     0.70

### Subject Summary

Compute metrics per subject.

In [16]:
grps = tm_dfw.df.groupby(level=['tst_dataset', 'tst_subject','dataset', 'subject'])

grps.mean()

bACC       ACC       TPR  \
tst_dataset tst_subject dataset subject                                 
BGSInt      1           BGSInt  1        0.822272  0.859953  0.783877   
                        BGSObs  1        0.743836  0.856773  0.627636   
                        OAObs   1        0.748331  0.843305  0.651885   
                        OAOut   1        0.827934  0.826176  0.829700   
            2           BGSInt  2        0.705356  0.794772  0.613438   
                        BGSObs  2        0.623005  0.801971  0.439485   
                        OAObs   2        0.618783  0.783370  0.451844   
                        OAOut   2        0.704965  0.760275  0.649000   
            3           BGSInt  3        0.768790  0.848526  0.686970   
                        BGSObs  3        0.664785  0.871925  0.452451   
                        OAObs   3        0.699392  0.822710  0.574553   
                        OAOut   3        0.780529  0.781574  0.779628   
            4           BGSInt  4        0.702754  0.729314  0.675761   
                        BGSObs  4        0.569846  0.648446  0.489255   
                        OAObs   4        0.614042  0.712318  0.514489   
                        OAOut   4        0.779381  0.711617  0.848105   
            5           BGSInt  5        0.888581  0.924166  0.852181   
                        BGSObs  5        0.746080  0.950354  0.536340   
                        OAObs   5        0.565316  0.931567  0.194830   
                        OAOut   5        0.719112  0.908097  0.528672   
            6           BGSInt  6        0.820042  0.855529  0.783805   
                        BGSObs  6        0.638733  0.881361  0.389232   
                        OAObs   6        0.633191  0.886608  0.376559   
                        OAOut   6        0.772646  0.860580  0.681830   
            7           BGSInt  7        0.892804  0.872889  0.912982   
                        BGSObs  7        0.652361  0.837913  0.462150   
                        OAObs   7        0.488475  0.901048  0.071015   
                        OAOut   7        0.739557  0.903232  0.573138   
            8           BGSInt  8        0.762002  0.790061  0.733500   
                        BGSObs  8        0.650389  0.816208  0.480089   
                        OAObs   8        0.649550  0.799058  0.498081   
                        OAOut   8        0.834908  0.784902  0.885341   
            9           BGSInt  9        0.653702  0.891430  0.411060   
                        BGSObs  9        0.592345  0.876711  0.299347   
                        OAObs   9        0.608224  0.858916  0.354872   
                        OAOut   9        0.779628  0.866535  0.691717   
BGSObs      1           BGSInt  1        0.740623  0.819519  0.660257   
                        BGSObs  1        0.758217  0.845472  0.668534   
                        OAObs   1        0.654114  0.805284  0.500705   
                        OAOut   1        0.725132  0.798641  0.650932   
            2           BGSInt  2        0.661875  0.718617  0.603745   
                        BGSObs  2        0.667268  0.738152  0.594305   
                        OAObs   2        0.556703  0.705388  0.405924   
                        OAOut   2        0.626822  0.693437  0.559722   
            3           BGSInt  3        0.759333  0.797829  0.719740   
                        BGSObs  3        0.751728  0.874042  0.626157   
                        OAObs   3        0.697848  0.734153  0.661023   
                        OAOut   3        0.754239  0.690946  0.818359   
            4           BGSInt  4        0.581029  0.645704  0.515051   
                        BGSObs  4        0.641369  0.649560  0.632854   
                        OAObs   4        0.583975  0.633617  0.533669   
                        OAOut   4        0.618070  0.483282  0.754922   
            5           BGSInt  5        0.833311  0.823957  0.842853   
                        BGSObs

### Average Across Subjects

Compute metrics across all subjects and trials.

In [17]:
grps = tm_dfw.loc('bACC', 'TPR', 'TNR').groupby(level=['tst_dataset', 'dataset'])

grps.describe()

bACC                                                    \
                     count      mean       std       min       25%       50%   
tst_dataset dataset                                                            
BGSInt      BGSInt    90.0  0.779589  0.097825  0.538147  0.714247  0.787987   
            BGSObs   900.0  0.653487  0.091613  0.419643  0.587556  0.648235   
            OAObs    900.0  0.625034  0.101525  0.426875  0.540144  0.624585   
            OAOut    900.0  0.770962  0.092797  0.443466  0.724150  0.785348   
BGSObs      BGSInt   900.0  0.719500  0.100985  0.428323  0.650149  0.721246   
            BGSObs    90.0  0.708666  0.084016  0.502105  0.652998  0.704149   
            OAObs    900.0  0.619188  0.087709  0.391826  0.558504  0.613174   
            OAOut    900.0  0.707339  0.109647  0.203831  0.640350  0.717278   
OAObs       BGSInt   900.0  0.617804  0.077212  0.422874  0.560743  0.609901   
            BGSObs   900.0  0.569720  0.063753  0.411305  0.523771  0.561577   
            OAObs     90.0  0.699988  0.099039  0.486080  0.633401  0.706799   
            OAOut    900.0  0.793642  0.100527  0.458262  0.724079  0.801221   
OAOut       BGSInt   900.0  0.625444  0.098380  0.437006  0.551626  0.609881   
            BGSObs   900.0  0.557923  0.076204  0.435492  0.507366  0.536937   
            OAObs    900.0  0.617506  0.087508  0.445965  0.552421  0.611876   
            OAOut     90.0  0.867439  0.083729  0.633702  0.826015  0.881266   

                                           TPR            ...            \
                          75%       max  count      mean  ...       75%   
tst_dataset dataset                                       ...             
BGSInt      BGSInt   0.845705  0.953157   90.0  0.717064  ...  0.844615   
            BGSObs   0.715301  0.931335  900.0  0.463998  ...  0.583333   
            OAObs    0.699378  0.909065  900.0  0.409792  ...  0.583333   
            OAOut    0.835409  0.972412  900.0  0.718570  ...  0.875000   
BGSObs      BGSInt   0.797198  0.937883  900.0  0.676645  ...  0.809524   
            BGSObs   0.771760  0.884165   90.0  0.632111  ...  0.735965   
            OAObs    0.681752  0.910096  900.0  0.495540  ...  0.625000   
            OAOut    0.791056  0.924630  900.0  0.696905  ...  0.857143   
OAObs       BGSInt   0.673227  0.869278  900.0  0.366962  ...  0.471362   
            BGSObs   0.609568  0.794649  900.0  0.259881  ...  0.366667   
            OAObs    0.762002  0.937394   90.0  0.540878  ...  0.700000   
            OAOut    0.864702  0.993458  900.0  0.740161  ...  0.875000   
OAOut       BGSInt   0.679088  0.913337  900.0  0.330783  ...  0.451136   
            BGSObs   0.581593  0.860030  900.0  0.180569  ...  0.238095   
            OAObs    0.671376  0.926294  900.0  0.297184  ...  0.416667   
            OAOut    0.930724  0.989916   90.0  0.800848  ...  0.941667   

                                 TNR                                          \
                          max  count      mean       std       min       25%   
tst_dataset dataset                                                            
BGSInt      BGSInt   1.000000   90.0  0.842114  0.072683  0.572939  0.803410   
            BGSObs   1.000000  900.0  0.842975  0.095771  0.469066  0.808167   
            OAObs    1.000000  900.0  0.840275  0.082333  0.548835  0.793087   
            OAOut    1.000000  900.0  0.823355  0.083416  0.549601  0.770303   
BGSObs      BGSInt   1.000000  900.0  0.762355  0.081941  0.473793  0.713764   
            BGSObs   1.000000   90.0  0.785222  0.088474  0.532995  0.738799   
            OAObs    1.000000  900.0  0.742836  0.086567  0.490869  0.682884   
            OAOut    1.000000  900.0  0.717773  0.124949  0.135074  0.662736   
OAObs       BGSInt   0.857143  900.0  0.868647  0.066719  0.667720  0.829331   
            BGSObs   0.750000  900.0  0.879560  0.073267  0.638422  0.837924   
            OAObs    1.000000   90.

# Significance Testing

This section contains post-hoc test related code.

`LoadExperimentResults` servers as a class which will load all 'test-metrics.csv' files found in a given directory. This acts similar to the code we saw above but is now given in an OOP format.

In [18]:
class LoadExperimentResults():
    rename_cols = {
        'tpr': 'TPR',
        'tnr': 'TNR',
        'ppv': 'PPV',
        'nll': 'NLL'
    }
    convert_type = {
        'subject': str,
        'trial': str,
    }
    exp_cols = [
        'tst_dataset', 
        'tst_subject', 
        'tst_trial'
    ]
    mi = [
        'tst_dataset', 
        'tst_subject', 
        'tst_trial', 
        'dataset',
        'subject',
        'trial'
    ]

    def __init__(self, subjects=9, tasks=4):      
        self.subjects = subjects 
        self.tasks = tasks

    def __call__(self, target_exp_sub_dirs):
        target_file = '**/test-metrics.csv'
        parent_dir = Path(ds.ROOT_DIR) / ds.EXPS_DIR
        exps = {}
        for name, target_dir in target_exp_sub_dirs.items():
            target_path = parent_dir / target_dir / target_file
            print(f"Loading files frome {target_path}")
            files = glob.glob(str(target_path), recursive=True)

            print(f"Found {len(files)} files")
            self._check_found_file_count(files)

            exps[name] = self._load_files(files) 
            
        return exps

    def _check_found_file_count(self, files):
        estimated_files = self.subjects * self.tasks
        assert estimated_files == len(files), f"Number of files found {len(files)} does not match estimated {estimated_files}"
    
    def _load_files(self, files):
        list_dfs = []
        for f in files:
            df = pd.read_csv(f)
            self._convert_types(df)
            df = self._parse_exp(df)
            df.rename(columns=self.rename_cols, inplace=True)
            list_dfs.append(df)
        df = pd.concat(list_dfs)
        dfm = df.set_index(self.mi).sort_index()
        return DataFrameIndexWrapper(dfm)
    
    def _parse_exp(self, df):
        exp_values = df.exp.str.split("-", expand=True)

        for col, index in zip(self.exp_cols[::-1], range(len(self.exp_cols))[::-1]):
            df.insert(0, col, exp_values[index])

        df.drop('exp', axis=1, inplace=True)
        
        return df

    def _convert_types(self, df):
        for col, t in self.convert_type.items():
            df[col] = df[col].astype(t)
        return df

## Loading and Directory Structure

Below is the directory structure that will be generated. All pickled files containing the statistical analysis results will be stored a directory named in accordance to the `current_analysis` variable. The directory created using the `current_analysis` variable will be a sub-directory of the `analysis/` directory (given by `ds.ANALYSIS_DIR`). 

The `target_exp_sub_dirs` variable is used to specify all the experiment names and their corresponding directories which contain the results. The key is the name of the experiment (anything you want) while the value is the path to the experiment specific experiment which is a sub-directory of the directory given by `current_analysis`.

In [19]:
target_exp_sub_dirs = {
    'eegnet':os.path.join(current_analysis, 'logocv-v2-eegnet-async-upsample'),
    'svm':os.path.join(current_analysis,'logocv-v2-svm-async'),
}

In [20]:

# DO NOT edit the below code unless you know what you are doing!
# Otherwise you might break the directory structure and things might
# further break later on.
analysis_path = os.path.join(ds.ROOT_DIR, ds.ANALYSIS_DIR, current_analysis)
pickles_path = os.path.join(ds.ROOT_DIR, ds.ANALYSIS_DIR, current_analysis, 'pickles')

exps_pickle_path = os.path.join(pickles_path, 'exps.pkl')
bl_pickle_path = os.path.join(pickles_path, 'baseline_analysis.pkl')
bl_multi_pickle_path = os.path.join(pickles_path, 'baseline_multi_analysis.pkl')
same_target_pickle_path = os.path.join(pickles_path, 'same_target_analysis.pkl')
same_source_pickle_path = os.path.join(pickles_path, 'same_source_analysis.pkl')
within_task_pickle_path = os.path.join(pickles_path, 'within_task_analysis.pkl')
wthin_category_pickle_path = os.path.join(pickles_path, 'within_category_analysis.pkl')

if not os.path.exists(analysis_path): os.makedirs(analysis_path)
if not os.path.exists(pickles_path): os.makedirs(pickles_path)

ler = LoadExperimentResults()
exps = ler(target_exp_sub_dirs)
print(f"\nLoaded experiments:\n{exps}")


with open(exps_pickle_path, 'wb') as file:
    pickle.dump(exps, file)

Loading files frome /home/dev/mnt/scripts/classification/exps/v2-ErrP-variations-March-3-2022/logocv-v2-eegnet-async-upsample/**/test-metrics.csv
Found 36 files
Loading files frome /home/dev/mnt/scripts/classification/exps/v2-ErrP-variations-March-3-2022/logocv-v2-svm-async/**/test-metrics.csv
Found 36 files

Loaded experiments:
{'eegnet': DataFrameIndexWrapper(shape=(11160, 9), 'svm': DataFrameIndexWrapper(shape=(11160, 6)}


In [21]:
exps['eegnet'].df

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
BGSInt      1           1         BGSInt  1       1      0.836468  0.886538   
                                  BGSObs  1       1      0.765794  0.797729   
                                                  10     0.729411  0.846679   
                                                  2      0.801884  0.842515   
                                                  3      0.699750  0.877126   
...                                                           ...       ...   
OAOut       9           9         OAObs   9       6      0.497313  0.989311   
                                                  7      0.544373  0.992477   
                                                  8      0.556045  0.983396   
                                                  9      0.537899  0.986631   
                                  OAOut   9       9      0.844737  0.987879   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
BGSInt      1           1         BGSInt  1       1      0.785714  0.045082   
                                  BGSObs  1       1      0.733333  0.028871   
                                                  10     0.608696  0.055556   
                                                  2      0.760000  0.068841   
                                                  3      0.516129  0.071111   
...                                                           ...       ...   
OAOut       9           9         OAObs   9       6      0.000000  0.000000   
                                                  7      0.090909  0.200000   
                                                  8      0.125000  0.040000   
                                                  9      0.083333  0.066667   
                                  OAOut   9       9      0.700000  0.269231   

                                                              TNR        F1  \
tst_dataset tst_subject tst_trial dataset subject trial                       
BGSInt      1           1         BGSInt  1       1      0.887222  0.085271   
                                  BGSObs  1       1      0.798255  0.055556   
                                                  10     0.850126  0.101818   
                                                  2      0.843769  0.126246   
                                                  3      0.883371  0.125000   
...                                                           ...       ...   
OAOut       9           9         OAObs   9       6      0.994627  0.000000   
                                                  7      0.997838  0.125000   
                                                  8      0.987090  0.060606   
                                                  9      0.992465  0.074074   
                                  OAOut   9       9      0.989474  0.388889   

                                                              NLL        PR  \
tst_dataset tst_subject tst_trial dataset subject trial                       
BGSInt      1           1         BGSInt  1       1      0.279351  0.372065   
                                  BGSObs  1       1      0.442184  0.140017   
                                                  10     0.352951  0.082405   
                                                  2      0.306565  0.165552   
                                                  3      0.302363  0.181567   
...                                                           ...       ...   
OAOut       9           9         OAObs   9       6      0.043309  0.023518   
                                                  7      0.034934  0.171876   
                                                  8      0.056152  0.029727   
                                                  9      0.058268  0.020794   
                             

In [22]:
exps['svm'].df

bACC       ACC  \
tst_dataset tst_subject tst_trial dataset subject trial                       
BGSInt      1           1         BGSInt  1       1      0.717121  0.790385   
                                  BGSObs  1       1      0.684406  0.767442   
                                                  10     0.631776  0.738672   
                                                  2      0.697508  0.753293   
                                                  3      0.671236  0.789907   
...                                                           ...       ...   
OAOut       9           9         OAObs   9       6      0.533342  0.962052   
                                                  7      0.704570  0.951639   
                                                  8      0.656031  0.934655   
                                                  9      0.513141  0.937433   
                                  OAOut   9       9      0.926177  0.952066   

                                                              TPR       PPV  \
tst_dataset tst_subject tst_trial dataset subject trial                       
BGSInt      1           1         BGSInt  1       1      0.642857  0.020455   
                                  BGSObs  1       1      0.600000  0.020785   
                                                  10     0.521739  0.028436   
                                                  2      0.640000  0.038186   
                                                  3      0.548387  0.044041   
...                                                           ...       ...   
OAOut       9           9         OAObs   9       6      0.100000  0.015873   
                                                  7      0.454545  0.056180   
                                                  8      0.375000  0.025000   
                                                  9      0.083333  0.009346   
                                  OAOut   9       9      0.900000  0.094737   

                                                              TNR        F1  
tst_dataset tst_subject tst_trial dataset subject trial                      
BGSInt      1           1         BGSInt  1       1      0.791384  0.039648  
                                  BGSObs  1       1      0.768811  0.040179  
                                                  10     0.741814  0.053933  
                                                  2      0.755015  0.072072  
                                                  3      0.794085  0.081535  
...                                                           ...       ...  
OAOut       9           9         OAObs   9       6      0.966685  0.027397  
                                                  7      0.954595  0.100000  
                                                  8      0.937063  0.046875  
                                                  9      0.942949  0.016807  
                                  OAOut   9       9      0.952355  0.171429  

[11160 rows x 6 columns]

## R Wrapper
In order to conduct many of the post-hoc tests we utilize R code. This is done by a Python R wrapper `rpy2`.

### Install R Packages

Run the `install_r.py` script from the command line to automatically install all R packages into the current conda environment. If you don't install these packages then the following code will fail to run!

WARNING: Running the `install_r.py` script from within the notebook tends to fail!

In [23]:
import rpy2
import rpy2.robjects as ro
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr, isinstalled
from rpy2.robjects.conversion import localconverter
import rpy2.rinterface as ri

### RHypothesisTests

This class contains the `r2py` code for running statistical tests in R using Python.

https://rpy2.github.io/doc/latest/html/introduction.html

In [24]:
class RHypothesisTests():
    """ Computes R stats analysis and post-hoc test.
    
        All tests are focused on unequal variance between groups.
    """
    es = importr('effectsize')
    rstat = importr('rstatix')
    base = importr('base')
    boot = importr('boot')
    
    def __init__(self,  alpha=0.95):
        self.as_dataframe = r('as.data.frame')
        self.alpha = alpha
        self.conf_level = 1-alpha
    
    def welch_test(self, df):
        """ Compute Welch's t-test.
        
            Ref: https://www.rdocumentation.org/packages/rstatix/versions/0.7.0/topics/t_test
        
        """
        fml = ro.Formula('score ~ group')
        
        rdf = self.rstat.t_test(
            formula=fml, 
            data=self.df_to_r(df), 
            var_equal=False, 
            alternative = "two.sided",
            conf_level=self.conf_level,
            detailed=True
        )
        return self.r_to_df(rdf).reset_index(drop=True)

    def games_howell(self, df):
        """ Computes Games Howell Post-hoc with Welch's DoF correction.
        
            Ref: https://rpkgs.datanovia.com/rstatix/reference/games_howell_test.html
        
        """
        fml = ro.Formula('score ~ group')
        rdf = self.rstat.games_howell_test(
            formula=fml, 
            data=self.df_to_r(df),
            conf_level=self.conf_level,
            detailed=True
        )
        return  self.r_to_df(rdf).reset_index(drop=True)
    
    def hedges_g_star(self, df):
        """ Computes Hedges' g* by disabling pooled_sd and computing mean SD
            from both groups. 
            
            Ref: https://easystats.github.io/effectsize/reference/cohens_d.html
        """
        fml = ro.Formula('score ~ group')
        rdf = self.es.hedges_g(
            x=fml, 
            data=self.df_to_r(df),
            pooled_sd=False,
            ci=0.95,
            alternative="two.sided"
        )
        return self.r_to_df(rdf).reset_index(drop=True)

    def mean_info(self, df, forward=True):
        """ Compute mean related stats and adds them to the passed DataFrame """
        if len(df['group'].unique()) != 2:
            err = "More than two unique groups were detected in 'group' column."
            raise ValueError(err)

        columns = ['mean1', 'mean2', 'std1', 'std2', 'diff']
        means = df.groupby('group').mean()['score'].values
        
        diff = means[0] - means[1] if forward else  means[1] - means[0]
        std =  df.groupby('group').std(ddof=1)['score'].values
        return pd.DataFrame(np.hstack([means, std, diff]).reshape(1, -1), columns=columns)
       
    def df_to_r(self, df):
        """ Converts a Python DataFrame to a R DataFrame"""
        with localconverter(ro.default_converter + pandas2ri.converter):
            rdf = ro.conversion.py2rpy(df)
        return rdf
    
    def r_to_df(self, rdf):
        """ Converts a R DataFrame to a Python DataFrame"""
        with localconverter(ro.default_converter + pandas2ri.converter):
            df = ro.conversion.rpy2py(rdf)
        return df

### ExperimentAnaylsis

This class utilizes `RHypothesisTests` to run the post-hoc tests. The `baseline()`, `same_source()`, `same_target()`, `within_task()` and `within_category()` methods correspond to the baseline, same source, same target, within task and within ErrP category experiments ran in the paper. 

In [25]:
def get_endswith(df, string):
    """ Get all the elements whose index starts with a given string """
    return df[df.index.str.endswith(string)]

def get_startswith(df, string):
    """ Get all the elements whose index ends with a given string """
    return df[df.index.str.startswith(string)]

def get_equals(df, string):
    """ Get all the elements whose index is equal to a given string """
    return df.loc[string]

class ExperimentAnalysis():
    BASELINES = [
            ['BGSObs-BGSObs', 'BGSInt-BGSInt'],
            ['OAOut-OAOut', 'OAObs-OAObs']
        ]
    
    BASELINES_ALL = [
        [
            'BGSObs-BGSObs', 
            'BGSInt-BGSInt',
            'OAOut-OAOut', 
            'OAObs-OAObs',
        ]
    ]

    WITHIN_TASK  = [
        ['BGSInt-BGSObs', 'BGSObs-BGSInt'],
        ['OAOut-OAObs', 'OAObs-OAOut']
    ]

    WITHIN_CATEGORY = [
        ['BGSInt-OAOut', 'OAOut-BGSInt'],
        ['BGSObs-OAObs', 'OAObs-BGSObs']
    ]

    TASKS = ['BGSInt','BGSObs','OAOut','OAObs']
    
    def __init__(
        self, 
        exps, 
        alpha=.05, 
        level=['tst_dataset', 'dataset'],
        track_means=False,
    ):
        self.exps = exps
        self.level = level
        self.alpha = alpha
        self.rht = RHypothesisTests(alpha=alpha)
        
        self._tracker_status = dict(
            means=track_means
        )
        self._tracks = dict(
            means=None
        )
        
    def _run_analysis(self, dependent_vars, analysis_method, analysis_kwargs=None):
        """ Runs experiments for all classifiers and scores (i.e., dependent variables) 
        
            Each classifier has its own DataFrame and each dependent variable or score
            has its own sub-DataFrame or group that can be extracted.
            
            dependent_vars (List[str]): Name of scores/metrics to be used for analysis
            
            analysis_method (Callable): Reference to analysis method to be used.
                Either `self._multi_analysis` or `self._pair_analysis`.
                
            analysis_kwargs (dict): Kwargs for the corresponding method passed to 
                analysis_method.
        """
        analysis_kwargs = {} if analysis_kwargs is None else analysis_kwargs
        analysis_df =[]
        for exp, dfm in tqdm(self.exps.items()):
            dv_results = {}
            for dv in dependent_vars:
                df = dfm.get_group_data(level=self.level, score=dv)
                adf = analysis_method(df, **analysis_kwargs)
                adf['exps'], adf['score'] = exp, dv
                mi_names = ['exps', 'score']
                if adf.index.name is not None:
                    adf[adf.index.name] = adf.index
                    mi_names.append(adf.index.name)
                adf.set_index(mi_names, inplace=True)
                analysis_df.append(adf)
        analysis_df = pd.concat(analysis_df)
        analysis_df.sort_index(inplace=True)
        
        return analysis_df
    
    def baseline(self, dependent_vars, compare_all=False):
        """ Runs baselines experiments for either pair-wise or multi-analysis 
        
            dependent_vars (List[str]): Name of scores/metrics to be used for analysis
            
            compare_all (bool): Determines whether to use pair-wise or multi-analysis
        """
        if not compare_all:
            return self._run_analysis(
                dependent_vars, 
                analysis_method=self._pair_analysis,
                analysis_kwargs=dict(pairs=self.BASELINES)
            )
        else:
            return self._run_analysis(
                dependent_vars, 
                analysis_method=self._multi_analysis,
                analysis_kwargs=dict(same='equals', task=self.BASELINES_ALL)
            )
    
    def same_source(self, dependent_vars):
        """ Runs the same source multi-analysis experiment 
        
            dependent_vars (List[str]): Name of scores/metrics to be used for analysis
        """
        return self._run_analysis(
            dependent_vars, 
            analysis_method=self._multi_analysis,
            analysis_kwargs=dict(same='source', task=self.TASKS)
        )
    
    def same_target(self, dependent_vars):
        """ Runs the same target multi-analysis experiment 
        
            dependent_vars (List[str]): Name of scores/metrics to be used for analysis
        """
        return self._run_analysis(
            dependent_vars, 
            analysis_method=self._multi_analysis,
            analysis_kwargs=dict(same='target', task=self.TASKS)
        )
    
    def within_task(self, dependent_vars):
        """ Runs the within task pair-wise experiment 
        
            dependent_vars (List[str]): Name of scores/metrics to be used for analysis
        """
        return self._run_analysis(
            dependent_vars, 
            analysis_method=self._pair_analysis,
            analysis_kwargs=dict(pairs=self.WITHIN_TASK)
        )
    
    def within_category(self, dependent_vars):
        """ Runs the within category pair-wise experiment 
        
            dependent_vars (List[str]): Name of scores/metrics to be used for analysis
        """
        return self._run_analysis(
            dependent_vars, 
            analysis_method=self._pair_analysis,
            analysis_kwargs=dict(pairs=self.WITHIN_CATEGORY)
        )
    
    def _multi_analysis(self, df, same, task):
        """ Runs experiments for multiple analysis and constructs results DataFrame """
        if same.lower() == 'target':
            get_data = get_endswith
        elif same.lower() == 'source':
            get_data = get_startswith
        elif same.lower() == 'equals':
            get_data = get_equals
        else:
            err = f"'same' must be either 'target' or 'source': received {same}"
            raise ValueError(err)
            
        results_df = []
        for t in task:
            ldf = get_data(df, string=t).reset_index()

            pvalue_df = self.rht.games_howell(ldf)
            pvalue_df.rename(columns={'statistic':'t'}, inplace=True)
            # swap high and low cl so negative means 2nd mean was greater
            new_low = -1*pvalue_df['conf.high']
            pvalue_df['conf.high'] = -1*pvalue_df['conf.low']
            pvalue_df['conf.low'] = new_low
            
            es_df, mi_df = [], []
            for grps in pvalue_df[['group1', 'group2']].values:
                sub_df = df.loc[grps].reset_index()
                # Effect and CI
                effect_size_df = self.rht.hedges_g_star(sub_df)
                es_df.append(effect_size_df)
                # Means and diff
                mean_info_df = self.rht.mean_info(sub_df, forward=True)
                mi_df.append(mean_info_df)
            es_df = pd.concat(es_df, ignore_index=True)
            mi_df = pd.concat(mi_df, ignore_index=True)

            task_result_df = pd.concat([
                pvalue_df[['p.adj.signif', 'p.adj', 'group1', 'group2']], 
                mi_df, 
                pvalue_df[['se', 'conf.low', 'conf.high', 'n1', 'n2', 'df', 't']],
                es_df,
            ], axis=1).round(3)
            
            results_df.append(task_result_df)
        
        # Build and format analysis dataframe
        results_df = pd.concat(results_df, axis=0)
        
        if same in ['source', 'target']:
            grp1 = results_df['group1'].str.split("-", expand=True)
            grp1.rename(columns={0:'source', 1:'target'}, inplace=True)
            results_df.set_index(grp1[same],  inplace=True)
            results_df.sort_index(inplace=True)
        
        return results_df
    
    def _pair_analysis(self, df, pairs):
        """ Run pair-wise experiements and constructs results DataFrame """
        results = []
        for pair in pairs:
            pair_df = df.loc[pair].reset_index()
            
            pvalue_df = self.rht.welch_test(pair_df)
            # Effect and CI
            es_df = self.rht.hedges_g_star(pair_df)
            # Means and diff
            mi_df = self.rht.mean_info(pair_df)

            results_df = pd.concat([
                pvalue_df[['group1', 'group2']], 
                mi_df, 
                pvalue_df[['conf.low', 'conf.high', 'n1', 'n2', 'df', 'p']],
                es_df,
            ], axis=1).round(3)
            
            results.append(results_df)
            
        return pd.concat(results)

### Plotting and Util Functions

In [26]:
def check_CIs(mdf, target, low, high):
    """ Check if any CIs are invliad (i.e., point estimate falls outside CIs)"""
    ci_bool = mdf[target].between(mdf[low], mdf[high])
    ci_valid = np.all(ci_bool)
    print(f"All point estiamtes for '{target}' fall within CIs: {ci_valid}")
    if not ci_valid:
        print("WARNING: Invalid CIs detected!")
        invalid = mdf[(~ci_bool)][['group1', 'group2', target, low, high, 'n1', 'n2']]
        print(f"Number of invalid samples: {len(invalid)}")
        print(f"Print invalid CIs:\n{invalid}")
        return invalid

<hr>

## Baselines
Below is the code for running and saving the results for the baseline pair-wise and multi-analysis experiments.

### Paired comparison

In [27]:
ea = ExperimentAnalysis(exps)

bl_anlys = ea.baseline(dependent_vars=['bACC', 'TPR', 'TNR', 'PPV'])
bl_anlys

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.20it/s]


group1         group2  mean1  mean2   std1   std2   diff  \
exps   score                                                                    
eegnet PPV    BGSInt-BGSInt  BGSObs-BGSObs  0.056  0.044  0.033  0.024  0.013   
       PPV      OAObs-OAObs    OAOut-OAOut  0.031  0.094  0.025  0.061 -0.064   
       TNR    BGSInt-BGSInt  BGSObs-BGSObs  0.842  0.785  0.073  0.088  0.057   
       TNR      OAObs-OAObs    OAOut-OAOut  0.859  0.934  0.081  0.042 -0.075   
       TPR    BGSInt-BGSInt  BGSObs-BGSObs  0.717  0.632  0.191  0.161  0.085   
       TPR      OAObs-OAObs    OAOut-OAOut  0.541  0.801  0.215  0.167 -0.260   
       bACC   BGSInt-BGSInt  BGSObs-BGSObs  0.780  0.709  0.098  0.084  0.071   
       bACC     OAObs-OAObs    OAOut-OAOut  0.700  0.867  0.099  0.084 -0.167   
svm    PPV    BGSInt-BGSInt  BGSObs-BGSObs  0.034  0.028  0.019  0.012  0.007   
       PPV      OAObs-OAObs    OAOut-OAOut  0.015  0.067  0.007  0.046 -0.052   
       TNR    BGSInt-BGSInt  BGSObs-BGSObs  0.786  0.716  0.063  0.056  0.069   
       TNR      OAObs-OAObs    OAOut-OAOut  0.788  0.925  0.067  0.040 -0.137   
       TPR    BGSInt-BGSInt  BGSObs-BGSObs  0.623  0.585  0.166  0.164  0.038   
       TPR      OAObs-OAObs    OAOut-OAOut  0.495  0.671  0.175  0.210 -0.175   
       bACC   BGSInt-BGSInt  BGSObs-BGSObs  0.705  0.651  0.097  0.093  0.054   
       bACC     OAObs-OAObs    OAOut-OAOut  0.642  0.798  0.083  0.103 -0.156   

              conf.low  conf.high  n1  n2       df      p  Hedges_g    CI  \
exps   score                                                                
eegnet PPV       0.004      0.021  90  90  161.427  0.004     0.433  0.95   
       PPV      -0.077     -0.050  90  90  118.031  0.000    -1.347  0.95   
       TNR       0.033      0.081  90  90  171.537  0.000     0.700  0.95   
       TNR      -0.094     -0.056  90  90  133.123  0.000    -1.155  0.95   
       TPR       0.033      0.137  90  90  172.931  0.001     0.479  0.95   
       TPR      -0.317     -0.203  90  90  167.727  0.000    -1.346  0.95   
       bACC      0.044      0.098  90  90  174.031  0.000     0.774  0.95   
       bACC     -0.194     -0.140  90  90  173.206  0.000    -1.818  0.95   
svm    PPV       0.002      0.011  90  90  149.633  0.005     0.420  0.95   
       PPV      -0.062     -0.042  90  90   93.708  0.000    -1.567  0.95   
       TNR       0.052      0.087  90  90  175.753  0.000     1.148  0.95   
       TNR      -0.153     -0.121  90  90  145.581  0.000    -2.478  0.95   
       TPR      -0.010      0.087  90  90  177.973  0.120     0.232  0.95   
       TPR      -0.232     -0.118  90  90  172.249  0.000    -0.903  0.95   
       bACC      0.026      0.082  90  90  177.612  0.000     0.563  0.95   
       bACC     -0.184     -0.129  90  90  170.213  0.000    -1.664  0.95   

              CI_low  CI_high  
exps   score                   
eegnet PPV     0.138    0.727  
       PPV    -1.682   -1.008  
       TNR     0.398    0.999  
       TNR    -1.476   -0.832  
       TPR     0.183    0.774  
       TPR    -1.669   -1.020  
       bACC    0.471    1.075  
       bACC   -2.164   -1.468  
svm    PPV     0.124    0.714  
       PPV    -1.931   -1.198  
       TNR     0.832    1.462  
       TNR    -2.882   -2.069  
       TPR    -0.060    0.524  
       TPR    -1.208   -0.595  
       bACC    0.265    0.859  
       bACC   -2.003   -1.322

In [28]:
check_CIs(bl_anlys, target='diff', low='conf.low', high='conf.high')
check_CIs(bl_anlys, target='Hedges_g', low='CI_low', high='CI_high')

All point estiamtes for 'diff' fall within CIs: True
All point estiamtes for 'Hedges_g' fall within CIs: True


In [29]:
with open(bl_pickle_path, 'wb') as file:
    pickle.dump(bl_anlys, file)

### Multiple comparison

In [30]:
ea = ExperimentAnalysis(exps)

bl_anlys = ea.baseline(
    dependent_vars=['bACC', 'TPR', 'TNR', 'PPV'], 
    compare_all=True
)

bl_anlys

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.09it/s]


p.adj.signif  p.adj         group1         group2  mean1  mean2  \
exps   score                                                                   
eegnet PPV              *  0.021  BGSInt-BGSInt  BGSObs-BGSObs  0.056  0.044   
       PPV           ****  0.000  BGSInt-BGSInt    OAObs-OAObs  0.056  0.031   
       PPV           ****  0.000  BGSInt-BGSInt    OAOut-OAOut  0.056  0.094   
       PPV             **  0.002  BGSObs-BGSObs    OAObs-OAObs  0.044  0.031   
       PPV           ****  0.000  BGSObs-BGSObs    OAOut-OAOut  0.044  0.094   
       PPV           ****  0.000    OAObs-OAObs    OAOut-OAOut  0.031  0.094   
       TNR           ****  0.000  BGSInt-BGSInt  BGSObs-BGSObs  0.842  0.785   
       TNR             ns  0.452  BGSInt-BGSInt    OAObs-OAObs  0.842  0.859   
       TNR           ****  0.000  BGSInt-BGSInt    OAOut-OAOut  0.842  0.934   
       TNR           ****  0.000  BGSObs-BGSObs    OAObs-OAObs  0.785  0.859   
       TNR           ****  0.000  BGSObs-BGSObs    OAOut-OAOut  0.785  0.934   
       TNR           ****  0.000    OAObs-OAObs    OAOut-OAOut  0.859  0.934   
       TPR             **  0.008  BGSInt-BGSInt  BGSObs-BGSObs  0.717  0.632   
       TPR           ****  0.000  BGSInt-BGSInt    OAObs-OAObs  0.717  0.541   
       TPR              *  0.011  BGSInt-BGSInt    OAOut-OAOut  0.717  0.801   
       TPR             **  0.008  BGSObs-BGSObs    OAObs-OAObs  0.632  0.541   
       TPR           ****  0.000  BGSObs-BGSObs    OAOut-OAOut  0.632  0.801   
       TPR           ****  0.000    OAObs-OAObs    OAOut-OAOut  0.541  0.801   
       bACC          ****  0.000  BGSInt-BGSInt  BGSObs-BGSObs  0.780  0.709   
       bACC          ****  0.000  BGSInt-BGSInt    OAObs-OAObs  0.780  0.700   
       bACC          ****  0.000  BGSInt-BGSInt    OAOut-OAOut  0.780  0.867   
       bACC            ns  0.921  BGSObs-BGSObs    OAObs-OAObs  0.709  0.700   
       bACC          ****  0.000  BGSObs-BGSObs    OAOut-OAOut  0.709  0.867   
       bACC          ****  0.000    OAObs-OAObs    OAOut-OAOut  0.700  0.867   
svm    PPV              *  0.027  BGSInt-BGSInt  BGSObs-BGSObs  0.034  0.028   
       PPV           ****  0.000  BGSInt-BGSInt    OAObs-OAObs  0.034  0.015   
       PPV           ****  0.000  BGSInt-BGSInt    OAOut-OAOut  0.034  0.067   
       PPV           ****  0.000  BGSObs-BGSObs    OAObs-OAObs  0.028  0.015   
       PPV           ****  0.000  BGSObs-BGSObs    OAOut-OAOut  0.028  0.067   
       PPV           ****  0.000    OAObs-OAObs    OAOut-OAOut  0.015  0.067   
       TNR           ****  0.000  BGSInt-BGSInt  BGSObs-BGSObs  0.786  0.716   
       TNR             ns  0.995  BGSInt-BGSInt    OAObs-OAObs  0.786  0.788   
       TNR           ****  0.000  BGSInt-BGSInt    OAOut-OAOut  0.786  0.925   
       TNR           ****  0.000  BGSObs-BGSObs    OAObs-OAObs  0.716  0.788   
       TNR           ****  0.000  BGSObs-BGSObs    OAOut-OAOut  0.716  0.925   
       TNR           ****  0.000    OAObs-OAObs    OAOut-OAOut  0.788  0.925   
       TPR             ns  0.402  BGSInt-BGSInt  BGSObs-BGSObs  0.623  0.585   
       TPR           ****  0.000  BGSInt-BGSInt    OAObs-OAObs  0.623  0.495   
       TPR             ns  0.345  BGSInt-BGSInt    OAOut-OAOut  0.623  0.671   
       TPR             **  0.003  BGSObs-BGSObs    OAObs-OAObs  0.585  0.495   
       TPR              *  0.014  BGSObs-BGSObs    OAOut-OAOut  0.585  0.671   
       TPR           ****  0.000    OAObs-OAObs    OAOut-OAOut  0.495  0.671   
       bACC           ***  0.001  BGSInt-BGSInt  BGSObs-BGSObs  0.705  0.651   
       bACC          ****  0.000  BGSInt-BGSInt    OAObs-OAObs  0.705  0.642   
       bACC          ****  0.000  BGSInt-BGSInt    OAOut-OAOut  0.705  0.798   
       bACC            ns  0.899  BGSObs-BGSObs    OAObs-OAObs  0.651  0.642   
       bACC          ****  0.000  BGSObs-BGSObs    OAOut-OAOut  0.651  0.798   
       bACC          ****  0.000    OAObs-OAObs    OAOut-OAOut  0.642  0.798   

            

In [31]:
with open(bl_multi_pickle_path, 'wb') as file:
    pickle.dump(bl_anlys, file)

## Transfer
Below is the code for running and saving the results for all transfer related experiments.

### Same Target (Test)

#### Multi-comparison

In [32]:
ea = ExperimentAnalysis(exps)

same_target_analysis = ea.same_target(dependent_vars=['bACC', 'TPR', 'TNR', 'PPV'])
same_target_analysis

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.55s/it]


p.adj.signif  p.adj         group1         group2  mean1  \
exps   score target                                                            
eegnet PPV   BGSInt         ****  0.000  BGSInt-BGSInt  BGSObs-BGSInt  0.056   
             BGSInt         ****  0.000  BGSInt-BGSInt   OAObs-BGSInt  0.056   
             BGSInt           ns  0.060  BGSInt-BGSInt   OAOut-BGSInt  0.056   
             BGSInt           ns  0.299  BGSObs-BGSInt   OAObs-BGSInt  0.034   
             BGSInt         ****  0.000  BGSObs-BGSInt   OAOut-BGSInt  0.034   
             BGSInt         ****  0.000   OAObs-BGSInt   OAOut-BGSInt  0.036   
             BGSObs           ns  0.389  BGSObs-BGSObs   OAOut-BGSObs  0.044   
             BGSObs            *  0.011  BGSObs-BGSObs   OAObs-BGSObs  0.044   
             BGSObs           ns  0.075   OAObs-BGSObs   OAOut-BGSObs  0.035   
             BGSObs         ****  0.000  BGSInt-BGSObs   OAObs-BGSObs  0.051   
             BGSObs            *  0.050  BGSInt-BGSObs  BGSObs-BGSObs  0.051   
             BGSObs         ****  0.000  BGSInt-BGSObs   OAOut-BGSObs  0.051   
             OAObs          ****  0.000   BGSObs-OAObs    OAOut-OAObs  0.013   
             OAObs          ****  0.000   BGSInt-OAObs   BGSObs-OAObs  0.017   
             OAObs          ****  0.000   BGSInt-OAObs    OAObs-OAObs  0.017   
             OAObs          ****  0.000   BGSInt-OAObs    OAOut-OAObs  0.017   
             OAObs          ****  0.000   BGSObs-OAObs    OAObs-OAObs  0.013   
             OAObs            ns  0.116    OAObs-OAObs    OAOut-OAObs  0.031   
             OAOut          ****  0.000   BGSInt-OAOut   BGSObs-OAOut  0.031   
             OAOut           ***  0.000   BGSInt-OAOut    OAObs-OAOut  0.031   
             OAOut          ****  0.000   BGSInt-OAOut    OAOut-OAOut  0.031   
             OAOut          ****  0.000   BGSObs-OAOut    OAObs-OAOut  0.019   
             OAOut          ****  0.000   BGSObs-OAOut    OAOut-OAOut  0.019   
             OAOut          ****  0.000    OAObs-OAOut    OAOut-OAOut  0.037   
       TNR   BGSInt         ****  0.000  BGSInt-BGSInt  BGSObs-BGSInt  0.842   
             BGSInt           **  0.007  BGSInt-BGSInt   OAObs-BGSInt  0.842   
             BGSInt         ****  0.000  BGSInt-BGSInt   OAOut-BGSInt  0.842   
             BGSInt         ****  0.000  BGSObs-BGSInt   OAObs-BGSInt  0.762   
             BGSInt         ****  0.000  BGSObs-BGSInt   OAOut-BGSInt  0.762   
             BGSInt         ****  0.000   OAObs-BGSInt   OAOut-BGSInt  0.869   
             BGSObs         ****  0.000  BGSObs-BGSObs   OAOut-BGSObs  0.785   
             BGSObs         ****  0.000  BGSObs-BGSObs   OAObs-BGSObs  0.785   
             BGSObs         ****  0.000   OAObs-BGSObs   OAOut-BGSObs  0.880   
             BGSObs         ****  0.000  BGSInt-BGSObs   OAObs-BGSObs  0.843   
             BGSObs         ****  0.000  BGSInt-BGSObs  BGSObs-BGSObs  0.843   
             BGSObs         ****  0.000  BGSInt-BGSObs   OAOut-BGSObs  0.843   
             OAObs          ****  0.000   BGSObs-OAObs    OAOut-OAObs  0.743   
             OAObs          ****  0.000   BGSInt-OAObs   BGSObs-OAObs  0.840   
             OAObs            ns  0.161   BGSInt-OAObs    OAObs-OAObs  0.840   
             OAObs          ****  0.000   BGSInt-OAObs    OAOut-OAObs  0.840   
             OAObs          ****  0.000   BGSObs-OAObs    OAObs-OAObs  0.743   
             OAObs          ****  0.000    OAObs-OAObs    OAOut-OAObs  0.859   
             OAOut          ****  0.000   BGSInt-OAOut   BGSObs-OAOut  0.823   
             OAOut          ****  0.000   BGSInt-OAOut    OAObs-OAOut  0.823   
             OAOut          ****  0.000   BGSInt-OAOut    OAOut-OAOut  0.823   
             OAOut          ****  0.000   BGSObs-OAOut    OAObs-OAOut  0.718   
             OAOut          ****  0.000   BGSObs-OAOut    OAOut-OAOut  0.718   
             OAOut          ****  0.000    OAObs-OAOut    OAOut-OAOut  0.847   
       TPR   BGSInt 

In [33]:
check_CIs(same_target_analysis, target='diff', low='conf.low', high='conf.high')
invalid = check_CIs(same_target_analysis, target='Hedges_g', low='CI_low', high='CI_high')

invalid

All point estiamtes for 'diff' fall within CIs: True
All point estiamtes for 'Hedges_g' fall within CIs: True


In [34]:
with open(same_target_pickle_path, 'wb') as file:
    pickle.dump(same_target_analysis, file)

### Same Source (Train)

#### Multi-comparison

In [35]:
ea = ExperimentAnalysis(exps)

same_source_analysis = ea.same_source(dependent_vars=['bACC', 'TPR', 'TNR', 'PPV'])
same_source_analysis

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.56s/it]


p.adj.signif  p.adj         group1         group2  mean1  \
exps   score source                                                            
eegnet PPV   BGSInt           ns  0.517  BGSInt-BGSInt  BGSInt-BGSObs  0.056   
             BGSInt         ****  0.000  BGSInt-BGSInt   BGSInt-OAObs  0.056   
             BGSInt         ****  0.000  BGSInt-BGSInt   BGSInt-OAOut  0.056   
             BGSInt         ****  0.000  BGSInt-BGSObs   BGSInt-OAObs  0.051   
             BGSInt         ****  0.000  BGSInt-BGSObs   BGSInt-OAOut  0.051   
             BGSInt         ****  0.000   BGSInt-OAObs   BGSInt-OAOut  0.017   
             BGSObs         ****  0.000  BGSObs-BGSObs   BGSObs-OAOut  0.044   
             BGSObs         ****  0.000  BGSObs-BGSObs   BGSObs-OAObs  0.044   
             BGSObs         ****  0.000   BGSObs-OAObs   BGSObs-OAOut  0.013   
             BGSObs         ****  0.000  BGSObs-BGSInt   BGSObs-OAObs  0.034   
             BGSObs          ***  0.001  BGSObs-BGSInt  BGSObs-BGSObs  0.034   
             BGSObs         ****  0.000  BGSObs-BGSInt   BGSObs-OAOut  0.034   
             OAObs            ns  0.589   OAObs-BGSObs    OAObs-OAOut  0.035   
             OAObs            ns  0.989   OAObs-BGSInt   OAObs-BGSObs  0.036   
             OAObs            ns  0.287   OAObs-BGSInt    OAObs-OAObs  0.036   
             OAObs            ns  0.720   OAObs-BGSInt    OAObs-OAOut  0.036   
             OAObs            ns  0.380   OAObs-BGSObs    OAObs-OAObs  0.035   
             OAObs            ns  0.122    OAObs-OAObs    OAObs-OAOut  0.031   
             OAOut          ****  0.000   OAOut-BGSInt   OAOut-BGSObs  0.047   
             OAOut          ****  0.000   OAOut-BGSInt    OAOut-OAObs  0.047   
             OAOut          ****  0.000   OAOut-BGSInt    OAOut-OAOut  0.047   
             OAOut            ns  0.612   OAOut-BGSObs    OAOut-OAObs  0.039   
             OAOut          ****  0.000   OAOut-BGSObs    OAOut-OAOut  0.039   
             OAOut          ****  0.000    OAOut-OAObs    OAOut-OAOut  0.037   
       TNR   BGSInt           ns  1.000  BGSInt-BGSInt  BGSInt-BGSObs  0.842   
             BGSInt           ns  0.996  BGSInt-BGSInt   BGSInt-OAObs  0.842   
             BGSInt           ns  0.104  BGSInt-BGSInt   BGSInt-OAOut  0.842   
             BGSInt           ns  0.919  BGSInt-BGSObs   BGSInt-OAObs  0.843   
             BGSInt         ****  0.000  BGSInt-BGSObs   BGSInt-OAOut  0.843   
             BGSInt         ****  0.000   BGSInt-OAObs   BGSInt-OAOut  0.840   
             BGSObs         ****  0.000  BGSObs-BGSObs   BGSObs-OAOut  0.785   
             BGSObs          ***  0.000  BGSObs-BGSObs   BGSObs-OAObs  0.785   
             BGSObs         ****  0.000   BGSObs-OAObs   BGSObs-OAOut  0.743   
             BGSObs         ****  0.000  BGSObs-BGSInt   BGSObs-OAObs  0.762   
             BGSObs           ns  0.093  BGSObs-BGSInt  BGSObs-BGSObs  0.762   
             BGSObs         ****  0.000  BGSObs-BGSInt   BGSObs-OAOut  0.762   
             OAObs          ****  0.000   OAObs-BGSObs    OAObs-OAOut  0.880   
             OAObs            **  0.005   OAObs-BGSInt   OAObs-BGSObs  0.869   
             OAObs            ns  0.702   OAObs-BGSInt    OAObs-OAObs  0.869   
             OAObs          ****  0.000   OAObs-BGSInt    OAObs-OAOut  0.869   
             OAObs            ns  0.104   OAObs-BGSObs    OAObs-OAObs  0.880   
             OAObs            ns  0.541    OAObs-OAObs    OAObs-OAOut  0.859   
             OAOut          ****  0.000   OAOut-BGSInt   OAOut-BGSObs  0.920   
             OAOut          ****  0.000   OAOut-BGSInt    OAOut-OAObs  0.920   
             OAOut             *  0.021   OAOut-BGSInt    OAOut-OAOut  0.920   
             OAOut            ns  0.671   OAOut-BGSObs    OAOut-OAObs  0.935   
             OAOut            ns  0.994   OAOut-BGSObs    OAOut-OAOut  0.935   
             OAOut            ns  0.843    OAOut-OAObs    OAOut-OAOut  0.938   
       TPR   BGSInt 

In [36]:
check_CIs(same_source_analysis, target='diff', low='conf.low', high='conf.high')
invalid = check_CIs(same_source_analysis, target='Hedges_g', low='CI_low', high='CI_high')

All point estiamtes for 'diff' fall within CIs: True
All point estiamtes for 'Hedges_g' fall within CIs: True


In [37]:
with open(same_source_pickle_path, 'wb') as file:
    pickle.dump(same_source_analysis, file)

## Within Task

### Paired comparison

In [38]:
ea = ExperimentAnalysis(exps)

within_task_analysis = ea.within_task(dependent_vars=['bACC', 'TPR', 'TNR', 'PPV'])
within_task_analysis

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.31it/s]


group1         group2  mean1  mean2   std1   std2   diff  \
exps   score                                                                    
eegnet PPV    BGSInt-BGSObs  BGSObs-BGSInt  0.051  0.034  0.040  0.018  0.017   
       PPV      OAObs-OAOut    OAOut-OAObs  0.037  0.037  0.029  0.033 -0.000   
       TNR    BGSInt-BGSObs  BGSObs-BGSInt  0.843  0.762  0.096  0.082  0.081   
       TNR      OAObs-OAOut    OAOut-OAObs  0.847  0.938  0.080  0.041 -0.091   
       TPR    BGSInt-BGSObs  BGSObs-BGSInt  0.464  0.677  0.176  0.180 -0.213   
       TPR      OAObs-OAOut    OAOut-OAObs  0.740  0.297  0.203  0.183  0.443   
       bACC   BGSInt-BGSObs  BGSObs-BGSInt  0.653  0.719  0.092  0.101 -0.066   
       bACC     OAObs-OAOut    OAOut-OAObs  0.794  0.618  0.101  0.088  0.176   
svm    PPV    BGSInt-BGSObs  BGSObs-BGSInt  0.029  0.022  0.015  0.010  0.006   
       PPV      OAObs-OAOut    OAOut-OAObs  0.022  0.024  0.013  0.018 -0.001   
       TNR    BGSInt-BGSObs  BGSObs-BGSInt  0.802  0.700  0.068  0.053  0.102   
       TNR      OAObs-OAOut    OAOut-OAObs  0.782  0.927  0.067  0.037 -0.146   
       TPR    BGSInt-BGSObs  BGSObs-BGSInt  0.408  0.602  0.151  0.166 -0.194   
       TPR      OAObs-OAOut    OAOut-OAObs  0.743  0.244  0.183  0.143  0.499   
       bACC   BGSInt-BGSObs  BGSObs-BGSInt  0.605  0.651  0.075  0.096 -0.046   
       bACC     OAObs-OAOut    OAOut-OAObs  0.762  0.586  0.089  0.068  0.177   

              conf.low  conf.high   n1   n2        df      p  Hedges_g    CI  \
exps   score                                                                   
eegnet PPV       0.014      0.020  900  900  1238.008  0.000     0.556  0.95   
       PPV      -0.003      0.003  900  900  1761.964  0.905    -0.006  0.95   
       TNR       0.072      0.089  900  900  1755.974  0.000     0.904  0.95   
       TNR      -0.097     -0.085  900  900  1338.990  0.000    -1.429  0.95   
       TPR      -0.229     -0.196  900  900  1796.919  0.000    -1.194  0.95   
       TPR       0.425      0.461  900  900  1777.770  0.000     2.291  0.95   
       bACC     -0.075     -0.057  900  900  1781.212  0.000    -0.684  0.95   
       bACC      0.167      0.185  900  900  1764.486  0.000     1.868  0.95   
svm    PPV       0.005      0.008  900  900  1524.886  0.000     0.499  0.95   
       PPV      -0.003      0.000  900  900  1667.806  0.111    -0.075  0.95   
       TNR       0.096      0.108  900  900  1698.664  0.000     1.674  0.95   
       TNR      -0.151     -0.141  900  900  1405.658  0.000    -2.708  0.95   
       TPR      -0.209     -0.180  900  900  1781.071  0.000    -1.226  0.95   
       TPR       0.484      0.514  900  900  1697.108  0.000     3.042  0.95   
       bACC     -0.054     -0.038  900  900  1692.487  0.000    -0.536  0.95   
       bACC      0.169      0.184  900  900  1677.201  0.000     2.231  0.95   

              CI_low  CI_high  
exps   score                   
eegnet PPV     0.461    0.651  
       PPV    -0.098    0.087  
       TNR     0.807    1.001  
       TNR    -1.536   -1.322  
       TPR    -1.294   -1.094  
       TPR     2.171    2.410  
       bACC   -0.779   -0.589  
       bACC    1.757    1.979  
svm    PPV     0.405    0.593  
       PPV    -0.168    0.017  
       TNR     1.565    1.782  
       TNR    -2.843   -2.571  
       TPR    -1.326   -1.125  
       TPR     2.904    3.179  
       bACC   -0.630   -0.441  
       bACC    2.111    2.350

In [39]:
check_CIs(within_task_analysis, target='diff', low='conf.low', high='conf.high')
check_CIs(within_task_analysis, target='Hedges_g', low='CI_low', high='CI_high')

All point estiamtes for 'diff' fall within CIs: True
All point estiamtes for 'Hedges_g' fall within CIs: True


In [40]:
with open(within_task_pickle_path, 'wb') as file:
    pickle.dump(within_task_analysis, file)

## Within ErrP Category

### Paired comparison

In [41]:
ea = ExperimentAnalysis(exps)

within_category_analysis = ea.within_category(dependent_vars=['bACC', 'TPR', 'TNR', 'PPV'])
within_category_analysis

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.43it/s]


group1        group2  mean1  mean2   std1   std2   diff  \
exps   score                                                                  
eegnet PPV    BGSInt-OAOut  OAOut-BGSInt  0.031  0.047  0.026  0.032 -0.016   
       PPV    BGSObs-OAObs  OAObs-BGSObs  0.013  0.035  0.006  0.030 -0.023   
       TNR    BGSInt-OAOut  OAOut-BGSInt  0.823  0.920  0.083  0.053 -0.097   
       TNR    BGSObs-OAObs  OAObs-BGSObs  0.743  0.880  0.087  0.073 -0.137   
       TPR    BGSInt-OAOut  OAOut-BGSInt  0.719  0.331  0.210  0.228  0.388   
       TPR    BGSObs-OAObs  OAObs-BGSObs  0.496  0.260  0.193  0.153  0.236   
       bACC   BGSInt-OAOut  OAOut-BGSInt  0.771  0.625  0.093  0.098  0.146   
       bACC   BGSObs-OAObs  OAObs-BGSObs  0.619  0.570  0.088  0.064  0.049   
svm    PPV    BGSInt-OAOut  OAOut-BGSInt  0.020  0.035  0.016  0.029 -0.015   
       PPV    BGSObs-OAObs  OAObs-BGSObs  0.008  0.020  0.004  0.010 -0.011   
       TNR    BGSInt-OAOut  OAOut-BGSInt  0.792  0.910  0.076  0.050 -0.118   
       TNR    BGSObs-OAObs  OAObs-BGSObs  0.709  0.792  0.063  0.075 -0.083   
       TPR    BGSInt-OAOut  OAOut-BGSInt  0.604  0.278  0.226  0.227  0.326   
       TPR    BGSObs-OAObs  OAObs-BGSObs  0.390  0.308  0.171  0.165  0.082   
       bACC   BGSInt-OAOut  OAOut-BGSInt  0.698  0.594  0.116  0.101  0.104   
       bACC   BGSObs-OAObs  OAObs-BGSObs  0.549  0.550  0.078  0.064 -0.001   

              conf.low  conf.high   n1   n2        df      p  Hedges_g    CI  \
exps   score                                                                   
eegnet PPV      -0.018     -0.013  900  900  1717.097  0.000    -0.535  0.95   
       PPV      -0.025     -0.021  900  900   979.152  0.000    -1.053  0.95   
       TNR      -0.103     -0.090  900  900  1526.678  0.000    -1.382  0.95   
       TNR      -0.144     -0.129  900  900  1750.189  0.000    -1.704  0.95   
       TPR       0.368      0.408  900  900  1786.349  0.000     1.767  0.95   
       TPR       0.220      0.252  900  900  1705.915  0.000     1.352  0.95   
       bACC      0.137      0.154  900  900  1791.898  0.000     1.521  0.95   
       bACC      0.042      0.057  900  900  1641.655  0.000     0.645  0.95   
svm    PPV      -0.017     -0.013  900  900  1370.694  0.000    -0.642  0.95   
       PPV      -0.012     -0.011  900  900  1198.605  0.000    -1.450  0.95   
       TNR      -0.124     -0.112  900  900  1543.700  0.000    -1.836  0.95   
       TNR      -0.090     -0.077  900  900  1749.055  0.000    -1.206  0.95   
       TPR       0.305      0.347  900  900  1797.949  0.000     1.439  0.95   
       TPR       0.066      0.097  900  900  1795.609  0.000     0.485  0.95   
       bACC      0.094      0.114  900  900  1766.264  0.000     0.953  0.95   
       bACC     -0.008      0.006  900  900  1729.954  0.777    -0.013  0.95   

              CI_low  CI_high  
exps   score                   
eegnet PPV    -0.628   -0.440  
       PPV    -1.156   -0.949  
       TNR    -1.487   -1.277  
       TNR    -1.812   -1.596  
       TPR     1.658    1.876  
       TPR     1.249    1.455  
       bACC    1.416    1.626  
       bACC    0.550    0.740  
svm    PPV    -0.737   -0.546  
       PPV    -1.559   -1.341  
       TNR    -1.949   -1.723  
       TNR    -1.307   -1.105  
       TPR     1.336    1.543  
       TPR     0.392    0.579  
       bACC    0.855    1.051  
       bACC   -0.106    0.079

In [42]:
check_CIs(within_category_analysis, target='diff', low='conf.low', high='conf.high')
check_CIs(within_category_analysis, target='Hedges_g', low='CI_low', high='CI_high')

All point estiamtes for 'diff' fall within CIs: True
All point estiamtes for 'Hedges_g' fall within CIs: True


In [43]:
with open(wthin_category_pickle_path, 'wb') as file:
    pickle.dump(within_category_analysis, file)

# Other Tests

Below is code for running other statistical checks.

In [44]:
# This is used to prevent the below tests when using the "Run All" cells option.
# Simply comment this out if you want these to run automatically.
raise NotImplementedError 

NotImplementedError: 

## Distribution Check

In [ ]:
%matplotlib qt
for exp_name, exp in exps.items():
    bacc_scores = exp.get_group_data(level=['tst_dataset', 'dataset'], score='bACC')
    groups = bacc_scores.groupby('group')
    fig, axes = plt.subplots(8,2, figsize=(20,15))
    for ax, (name, grp) in zip(axes.flatten(), groups):
        scores = grp.values.flatten()
        ax.hist(scores)

        title = "{}: Samples: {}".format(name, len(scores))
        ax.set_title(title)
    fig.supxlabel('score')
    fig.supylabel("number of smaples")
    fig.suptitle(exp_name)
    plt.tight_layout()
    plt.show();

## Normality

"Normality tests are useless on large datasets n>5000" https://stats.stackexchange.com/questions/2492/is-normality-testing-essentially-useless

In [ ]:
from scipy.stats import shapiro, probplot
%matplotlib qt

exps_norm = {}
p_thresh = 0.05
# source-target bACC scores grouping


for exp_name, exp in exps.items():
    bacc_scores = exp.get_group_data(level=['tst_dataset', 'dataset'], score='bACC')
    groups = bacc_scores.groupby('group')
    fig, axes = plt.subplots(8,2, figsize=(20,15))
    for ax, (name, grp) in zip(axes.flatten(), groups):
        scores = grp.values.flatten()
        stats, p = shapiro(scores)
        exps_norm[name] = p
        probplot(scores, dist="norm", plot=ax)

        if p > p_thresh:
            is_normal = True
        else:
            is_normal = False

        title = "{}: Samples: {}, p-value {:.2f}, Normal: {}".format(name, len(scores), p, is_normal)
        ax.set_title(title)
    fig.suptitle(exp_name)
    plt.tight_layout()
    plt.show();

## Equal Variance 

In [ ]:
from scipy.stats import bartlett, levene

def check_equal_variance(exps, test=bartlett, p_thresh=0.05):
    for exp_name, exp in exps.items():
        equal_var = []
        samples = {}
        print(f"Experiment: {exp_name}")
        bacc_scores = exp.get_group_data(level=['tst_dataset', 'dataset'], score='bACC')
        for name, grp in bacc_scores.groupby('group'):
            scores = grp.values.flatten()
            samples[name] = scores

        stats, p = bartlett(*samples.values())

        print("\tp-value: {:.4f}".format(p))
        if p > p_thresh:
            print("\tAccept Null - Assume Equal Variance")
        else:
            print("\tReject Null - Assume Non-Equal Variance")

In [ ]:
print(f"Variance test: bartlett")
check_equal_variance(exps, test=bartlett)

In [ ]:
print(f"Variance test: levene")
check_equal_variance(exps, test=levene)

## Collinearity 

In [ ]:
d1 = exps['eegnet'].df.loc[:, 'bACC'].values
d2 = exps['svm'].df.loc[:, 'bACC'].values

assert len(d1) == len(d2)

%matplotlib inline
plt.plot(d1, d2, '.b', )
plt.xlabel('eegnet bACC')
plt.ylabel('svm bACC')
plt.show()